In [1]:
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
with open('/content/drive/MyDrive/Build_Gpt/tiny_story.txt', 'r', encoding='utf-8') as f:
    text = f.read()

len(text)

5181596

In [3]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 128 # what is the maximum context length for predictions?
max_iters = 15000
eval_interval = 500
learning_rate = 1e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 256
n_head = 8
n_layer = 8
dropout = 0.1
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('/content/drive/MyDrive/Build_Gpt/tiny_story.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

print("len(train_data) - ", len(train_data))
print("len(val_data) - ", len(val_data))

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out


class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)


class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))

        return x


# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        tok_emb = self.token_embedding_table(idx)  #(B, T, n_embd(32))
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))   # (T, n_embd)
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)   # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))


len(train_data) -  4663436
len(val_data) -  518160
6.391899 M parameters
step 0: train loss 4.6934, val loss 4.6924
step 500: train loss 2.1615, val loss 2.1527
step 1000: train loss 1.7376, val loss 1.7212
step 1500: train loss 1.4863, val loss 1.4723
step 2000: train loss 1.3450, val loss 1.3356
step 2500: train loss 1.2628, val loss 1.2483
step 3000: train loss 1.1904, val loss 1.1696
step 3500: train loss 1.1382, val loss 1.1313
step 4000: train loss 1.1027, val loss 1.0928
step 4500: train loss 1.0708, val loss 1.0613
step 5000: train loss 1.0427, val loss 1.0322
step 5500: train loss 1.0162, val loss 1.0128
step 6000: train loss 0.9969, val loss 0.9911
step 6500: train loss 0.9777, val loss 0.9734
step 7000: train loss 0.9669, val loss 0.9559
step 7500: train loss 0.9530, val loss 0.9413
step 8000: train loss 0.9381, val loss 0.9344
step 8500: train loss 0.9281, val loss 0.9210
step 9000: train loss 0.9220, val loss 0.9147
step 9500: train loss 0.9072, val loss 0.9043
step 10000:

# Do Generation

In [9]:
starting_text = encode("Once upon a time")
starting_text

[37, 62, 51, 53, 1, 69, 64, 63, 62, 1, 49, 1, 68, 57, 61, 53]

In [10]:
starting_context = torch.tensor([starting_text], dtype=torch.long, device=device)
starting_context

tensor([[37, 62, 51, 53,  1, 69, 64, 63, 62,  1, 49,  1, 68, 57, 61, 53]],
       device='cuda:0')

In [15]:
print(decode(m.generate(starting_context, max_new_tokens=5000)[0].tolist()))

Once upon a time there was a mystery facs. It was a little girl named Lily. She liked to play in her dolls, but some fell. A deep of rice wind and determth a new grateful princess. One day, she saw a witch. She wanted to set the bush and went to the bird, something is the pink in the mapo. She saw her little fearly what 'Happening in the water!" Mommy said, "but always do not her!"

Jenny found a breat! She kept like so scaring bush interesting in the slide, "Hellow, que. I'lp you're so taste. I did usaway it. I drove you to play a huge explore."

The boy said, "I can't take them!" The voice felt like us. He was sad, but he didn't do.
 Sara saw something to get idea. Chose where that if the feel like he had.

When they believed, Jake musure skiled and played selfice for his heart. Suddenly, the bird saw a big car with a ice. He was so happy he put the ball. He saw his mom tother to get scars and one more. 

"That not is a myster!" 
Mom took it afraid, Daisy and her mom asked her memori

In [8]:
print(decode(m.generate(starting_context, max_new_tokens=500)[0].tolist()))

Story Start

One day, there was a little girl named Lily. She loved to eat pot of her pool and reads to look at the hairy.

One day, her mommy said she was in the minutes with a big, making it nto play. Kitty was not dangeround he left right again and she started to sing. It not didn't wasn't too sweet.

Tom that day, she went to the right bom. She told Zoomy and Max blaughed at her hair. He would repling and was so happy people hands one light. And the cleverful vheil had it!

Suddenly, Jack would not lea


# Save the Model

In [4]:
# torch.save(m, '/content/drive/MyDrive/Build_Gpt/model_6M.pth')

# Load the Model

In [18]:
loaded_model = torch.load('/content/drive/MyDrive/Build_Gpt/model_6M.pth')

In [19]:
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

6.391899 M parameters
